# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

**Описание данных** 
В нашем распоряжении данные о поведении клиентов, которые перешли на тарифы 'smart' и 'ultra'. Данные включают себя кол-во звонков, их продолжительность, кол-во смс, кол-во мегабайт использованного интернета в месяц для каждого пользователя и подключенный тариф.

**План работы**
* Открыть файл с данными и изучить его;
* Разделить исходные данные на обучающую, валидационную и тестовую выборки;
* Исследовать качество разных моделей, меняя гиперпараметры. Кратко написать выводы исследования;
* Проверить качество модели на тестовой выборке; 
* Проверить модель на вменяемость;

## Откроем и изучим файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.metrics import plot_confusion_matrix

In [18]:
try:
    forsen = pd.read_csv('C:/Users/Games/Downloads/users_behavior.csv')
except:
    forsen = pd.read_csv('/datasets/users_behavior.csv')

forsen.info()
display(forsen)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


В датасете 3214 строк, 5 столбцов с данными для каждого пользователя:
* кол-во звонков
* минуты совершённых звонков
* кол-во отправленных сообщений
* кол-во МБ использованного интернета
* тариф 

## Разделим данные на выборки

In [3]:
forsen_train, forsen_other = train_test_split(forsen, test_size=0.4, random_state=777)
forsen_valid, forsen_test = train_test_split(forsen_other, test_size=0.5, random_state=777)

features_train = forsen_train.drop(['is_ultra'], axis=1) 
target_train = forsen_train['is_ultra']

features_valid = forsen_valid.drop(['is_ultra'], axis=1)
target_valid = forsen_valid['is_ultra']

features_test = forsen_test.drop(['is_ultra'], axis=1)
target_test = forsen_test['is_ultra']

Выше мы разделили все данные на три выборки - тренировочную, валидационную и тестовую в соотношении 3:1:1.

In [4]:
RANDOM_STATE=1234

## Исследуйте модели

Исследуем три модели на тренировочной и валидационной выборках и проверим их качество, меняя гиперпараметры:

Начнем с самой точной модели - модели случайного леса. Будем менять кол-во деревьев и шаги.

Для этого напишем циклы и укажем разное кол-во деревьев для исследования.

In [5]:
best_model = None
best_result = 0
best_est = 0
for est in range(1, 11, 1):
    model = RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=est) 
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid) 
    if result > best_result:
        best_model = model
        best_result = result
        best_est = est

print("Accuracy наилучшей модели на валидационной выборке:", best_result, "наилучшее кол-во деревьев:", best_est)

Accuracy наилучшей модели на валидационной выборке: 0.7978227060653188 наилучшее кол-во деревьев: 10


In [6]:
best_model = None
best_result = 0
best_est = 0
for est in range(100, 201, 50):
    model = RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=est) 
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid) 
    if result > best_result:
        best_model = model
        best_result = result
        best_est = est

print("Accuracy наилучшей модели на валидационной выборке:", best_result, "наилучшее кол-во деревьев:", best_est)

Accuracy наилучшей модели на валидационной выборке: 0.7869362363919129 наилучшее кол-во деревьев: 200


In [7]:
best_model = None
best_result = 0
best_est = 0
for est in range(50, 101, 10):
    model = RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=est) 
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid) 
    if result > best_result:
        best_model = model
        best_result = result
        best_est = est

print("Accuracy наилучшей модели на валидационной выборке:", best_result, "наилучшее кол-во деревьев:", best_est)

Accuracy наилучшей модели на валидационной выборке: 0.7884914463452566 наилучшее кол-во деревьев: 70


Самая качественная модель получилась при кол-ве деревьев равному 10. Accuracy наилучшей модели на валидационной выборке: 0.7978.

Далее исследуем модель решающего дерева. Будем менять гиперпараметр максимальной глубины от 1 до 5.

In [8]:
best_model = None
best_depth = 0
best_result = 0
for depth in range(1,6):
    model= DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=depth)
    model.fit(features_train, target_train)
    valid_predictions = model.predict(features_valid)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
        
print('Наиболее подходящая глубина дерева =', best_depth, ', Качество модели:', best_result)

Наиболее подходящая глубина дерева = 3 , Качество модели: 0.7698289269051322


Самая подходящая максимальная глубина с наивысшей точностью - 3. Качество такой модели - 0.7698. \
Качество модели решающего дерева ожидаемо оказалось ниже, чем у модели случайного леса.

Теперь исследуем качество модели логистической регрессии. Будем менять гиперпараметр максимальной итерации.

In [9]:
best_model = None
best_max_iter = 0
best_result = 0
for iter in [100, 250, 500, 750, 1000]:
    model = LogisticRegression(random_state=RANDOM_STATE, solver='lbfgs', max_iter=iter)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result = result
        best_max_iter = iter

print('Наиболее подходящее кол-во итераций:', best_max_iter, ', Качество модели:', best_result)

Наиболее подходящее кол-во итераций: 100 , Качество модели: 0.7262830482115086


Как видим, точность модели не изменялась при различных трешхолдах макс. итераций - 0.7263 при всех пяти экспериментах. \
Качество модели оказалось самым низким из трёх.

Подведём итоги исследования различных моделей и оценки их качества. \ 
Самой точной, как и ожидалось, оказалась модель случайного леса с кол-м деревьев равному 10. Accuracy на валидационной выборке: 0.7978.  

На втором месте - модель решающего дерева со значением гиперпараметра максимальной глубины 3. Accuracy на валидационной выборке: 0.7698.  

Замыкает тройку модель логистической регрессии, точность которой не менялась от изменения гиперпараметра максимального кол-ва итераций: 0.7263 при всех пяти экспериментах.

## Проверьте модель на тестовой выборке

Возьмём самую точную модель с подходящими гиперпараметрами из исследования для проверки на тестовой выборке. \
Модель случайного леса с кол-м деревьев равному десяти. Проверим её на тестовой выборке.

In [10]:
model = RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=10) 
model.fit(features_train, target_train)

predictions_test = model.predict(features_test)
accuracy_test = accuracy_score(target_test, predictions_test)
result = model.score(features_valid, target_valid) 
print('Качество модели на валидационной выборке:', result)
print()
print('Качество модели на тестовой выборке:', accuracy_test)

Качество модели на валидационной выборке: 0.7978227060653188

Качество модели на тестовой выборке: 0.7807153965785381


Как видим, качество модели на тестовой выборке несколько ниже, чем на валидационной, однако значимого переобучения не обнаружено, точность ниже лишь на ~0.0171

## (бонус) Проверьте модели на адекватность

Попробуем проверить нашу модель на адекватность (sanity check), используя DummyClassifier:

In [11]:
dummy_clf = DummyClassifier(random_state=RANDOM_STATE)
dummy_clf.fit(features_train, target_train)
dummy_predictions = dummy_clf.predict(features_test)
accuracy_sane = accuracy_score(target_test, dummy_predictions)
print(accuracy_sane)


0.6702954898911353


In [12]:
dummy_clf = DummyClassifier(random_state=RANDOM_STATE, strategy='most_frequent')
dummy_clf.fit(features_train, target_train)
dummy_predictions = dummy_clf.predict(features_test)
accuracy_sane = accuracy_score(target_test, dummy_predictions)
print(accuracy_sane)


0.6702954898911353


### Общий вывод

**Общий вывод исследования**  
Проведя исследование, мы выяснили, что:  
* самой качественной моделью из трёх (случайный лес, решающее дерево, логистическая регрессия) в нашем случае оказалась модель случайного леса со значением гиперпараметра кол-во деревьев - 10. Accuracy модели на валидационной выборке: 0.7978.
* при проверке данной модели на тестовой выборке она показала качество чуть ниже, чем на валидационной - 0.7807 и 0.7978 соответственно. Значимого переобучения не обнаружено, точность ниже лишь на ~0.0171.
* также мы проверили модель на вменяемость, качество модели оказалось выше, чем качество, полученное при sanity check с помощью DummyClassifier (0.7807 и 0.6703 соответственно)